In [1]:
import wget
import time

В нашем проекте мы готовим обработанные датасеты из открытых данных портала "Трудвсем - Работа России". Они будут опубликованы на портале https://data-in.ru/ В первую очередь это датасеты вакансий и резюме. Но и другие справочники и общая статистика портала "Трудвсем". Они доступны здесь: https://trudvsem.ru/opendata/datasets

Кратко проблемы можно описать следующим образом: * файлы с интересующими нас данными очень большие (14Гб); * это xml-файлы.

С течением времени проект переформулировался в более амбициозную задачу - собрать все данные за 4 года, так как в каждом новом дампе (которые появляются на портале почти каждый день) старые резюме и вакансии удаляются, а они могли бы представлять интерес для исследователей. 

Мы работаем в Яндекс-датасфере, мощности которой предоставляются нашей организации бесплатно, однако это не решает проблем с перегрузкой памяти, долгой загрузкой. Необходимо находить решения по оптимизации процесса.

Здесь сразу хочу сказать, что задача по распараллеливанию процессов загрузки файлов в датасферу, разархивации, парсинга ещё только в планах, однако её необходимо будет решить для того, чтобы ежеквартальное обновление проходило быстро и гладко с минимальным ручным вмешательством.

Вот для примера сколько занимает скачивание дампа (не требующего разархивации в данном случае) в датасферу

In [2]:
start_time = time.time()
wget.download('http://opendata.trudvsem.ru/7710538364-cv/data-20211125T194522-structure-20161130T143000.xml', 'resumes.xml')
print("--- %s seconds ---" % (time.time() - start_time))

--- 501.4737253189087 seconds ---


Можем посмотреть небольшой кусок файла без скачивания целиком с помощью BeautifulSoup и urllib

In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import lxml

In [4]:
data = urlopen('http://opendata.trudvsem.ru/7710538364-cv/data-20211125T194522-structure-20161130T143000.xml').read(5000).decode('utf8')

In [5]:
soup = BeautifulSoup(data, 'lxml-xml')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<cvs vocab="http://schema.org/" xmlns:dc="http://purl.org/dc/terms/">
 <cv about="http://opendata.trudvsem.ru/7710538364-cv/cv.xml#652be920-6f77-11eb-b531-ef76bd2a03c1" typeof="Person">
  <region rel="dc:references" resource="http://opendata.trudvsem.ru/7710538364-regions/regions.xml#4200000000000"/>
  <profession rel="dc:references" resource="http://opendata.trudvsem.ru/7710538364-professions/professions.xml#129014"/>
  <industry rel="dc:references" resource="http://opendata.trudvsem.ru/7710538364-industries/industries.xml#Food"/>
  <positionName property="jobTitle">
   Кондитер
  </positionName>
  <creationDate>
   2021-02-15
  </creationDate>
  <locality>
   4200001200000
  </locality>
  <workExperienceList>
   <workExperience>
    <achievements>
     &lt;p&gt;&lt;em&gt;изготовление конфет ручной работы, зефир, маршмелоу&lt;/em&gt;&lt;/p&gt;
    </achievements>
    <idOwner>
     652be920-6f77-11eb-b531-ef76bd2a03c1
    </idOwner>
    <company_

Либо можем посмотреть кусок уже скаченного файла:

In [6]:
with open('resumes.xml', encoding='utf8') as fp:
    soup = BeautifulSoup(fp.read(1000), 'lxml-xml')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<cvs vocab="http://schema.org/" xmlns:dc="http://purl.org/dc/terms/">
 <cv about="http://opendata.trudvsem.ru/7710538364-cv/cv.xml#652be920-6f77-11eb-b531-ef76bd2a03c1" typeof="Person">
  <region rel="dc:references" resource="http://opendata.trudvsem.ru/7710538364-regions/regions.xml#4200000000000"/>
  <profession rel="dc:references" resource="http://opendata.trudvsem.ru/7710538364-professions/professions.xml#129014"/>
  <industry rel="dc:references" resource="http://opendata.trudvsem.ru/7710538364-industries/industries.xml#Food"/>
  <positionName property="jobTitle">
   Кондитер
  </positionName>
  <creationDate>
   2021-02-15
  </creationDate>
  <locality>
   4200001200000
  </locality>
  <workExperienceList>
   <workExperience>
    <achievements>
     &lt;p&gt;&lt;em&gt;изготовление конфет ручной работы, зефир, маршмелоу&lt;/em&gt;&lt;/p&gt;
    </achievements>
    <idOwner>
     652be920-6f77-11eb-b531-ef76bd2a03c1
    </idOwner>
    <company_

Для сбора в привычные пандасовские таблицы было принято решение использовать на Прекрасныйсуп, который просто не справляется с файлом такого объёма, а парсер lxml (которым Прекрасныйсуп и пользуется) с его методом iterparse. На первых этапах память тоже жёстко перегружалась и ядро умирало (точнее 4 или 8 ядер). 

Решением стали очистка пройденного элемента и удаление построенного "куска xml-дерева" из памяти после каждого распарсенного элемента.

    elem.clear()
    for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0] 

В примере ниже распарсим только 1000 записей резюме. Но в случае последовательного сбора словарей, представляющих каждую будущую строку датафрейма {переменная: значение} постепенно опять сталкиваемся с перегрузкой памяти (при количестве более 1 млн записей, а их в современных дампах порядка 5 млн). Решением стала выгрузка части распарсенных записей в csv (по 500тыс). В принципе, можно сразу в БД выгружать, но у нас довольно обширная предобработка, так что мы собирали датафрейм из отдельных csv-файлов, обрабатывали и грузили в БД.

In [19]:
from lxml import etree
import pandas as pd
from tqdm import tqdm
df = pd.DataFrame()
# bar = tqdm() # Для большого количества записей можем воспользоваться счётчиком, сейчас только 1000 записей
i = 1
csv_size = 500000
l = []

start_time = time.time()
# Проходим по каждому элементу в файле
for event, elem in etree.iterparse('resumes.xml', tag='cv', recover=True):
    d = {}
    # И формируем словарик где ключ - имя элемента, а значение текст этого элемента
    d['about'] = elem.attrib['about'].rsplit('#', maxsplit=1)[-1]
    for element in list(elem): # Проходимся по каждому элементу в наборе
        if element.tag == 'profession':
            d['profession_code'] = element.attrib['resource'].rsplit('#', maxsplit=1)[-1]
            continue
        elif element.tag == 'region':
            d['region_code'] = element.attrib['resource'].rsplit('#', maxsplit=1)[-1]
            continue
        elif element.tag == 'industry':
            d['industry_code'] = element.attrib['resource'].rsplit('#', maxsplit=1)[-1]
            continue
        elif element.tag == 'workExperienceList' or element.tag == 'educationList' or element.tag == 'additionalEducationList':
            continue
        
        elif len(list(element)) >= 1: # Если длина значений этого элемента больше или равна 1, то перед нами словарь
            for sub_element in list(element):# По которому тоже нужно пройтись
                if sub_element.text != None:
                    d[element.tag + '_' + sub_element.tag] = sub_element.text
                    if len(list(sub_element)) >= 1: # Если длина значений этого элемента больше или равна 1, то перед нами словарь
                        for sub_sub_element in list(sub_element):# По которому тоже нужно пройтись
                            d[element.tag + '_' + sub_element.tag + '_' + sub_sub_element.tag] = sub_sub_element.text 
        elif element.text != None:
            d[element.tag] = element.text 
        else:
            continue
            
    elem.clear()
    for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0] 
    l.append(d)# Добавим этот словарик в список
#     bar.update(n=1)# Увеличиваем счетчик на 1
    
    if len(l) == 1000:
        break

    if len(l) == csv_size: # Когда список станет достаточно большим
        df = pd.DataFrame(l)
        df.to_csv(f'../../../500000cv{i}.csv',index=False)
        break
        
        i = i + 1
        l = []# И очищаем список
        df = pd.DataFrame()# И очищаем датафрейм
print("--- %s seconds ---" % (time.time() - start_time))
# Не забыть сделать df из остатков и выгрузить его!
df = pd.DataFrame(l) 
# df.to_csv('../../../file_name.csv',index=False)

--- 0.13121461868286133 seconds ---


Можно прикинуть, что при количестве записей порядка 5 млн парсинг файла будет идти 10 минут. Добавьте время на загрузку, разархивацию, обработку, загрузку в БД и другие технические этапы.

In [20]:
df

,about,region_code,profession_code,industry_code,positionName,creationDate,locality,educationType,worldskills_inspection_status,country_country_name,...,innerInfo_status,innerInfo_visibility,innerInfo_dateModify,innerInfo_deleted,innerInfo_fullnessRate,driveLicenceList_driveLicences,skills,additionalSkills,otherInfo,addCertificates
0,652be920-6f77-11eb-b531-ef76bd2a03c1,4200000000000,129014,Food,Кондитер,2021-02-15,4200001200000,Среднее профессиональное,Не применимо,Российская Федерация,...,Одобрено,Видно всем,2021-02-16,false,69,NaN,NaN,NaN,NaN,NaN
1,963949c0-6b73-11eb-b25a-e736a3d3ed84,3800000000000,242363,Education,Младший воспитатель,2021-02-10,3801500500000,Среднее,Не применимо,Российская Федерация,...,Одобрено,Видно всем,2021-05-22,false,56,NaN,NaN,NaN,NaN,NaN
2,0e71d3e9-c5db-11e7-aa78-037acc02728d,2300000000000,203369,Sales,Главный бухгалтер/Заместитель главного бухгалтера,2017-11-10,2300000100000,Высшее,Не применимо,Российская Федерация,...,Одобрено,Видно всем,2021-06-21,false,77,[B],<p>Знание налогового и бухгалтерского учета на...,"<p>Внимательность, ответственность, коммуникаб...",<p>Знание программ: 1С Предприятие Бухгалтерия...,NaN
3,d2b470e0-9839-11ea-9806-736ab11edb0c,3400000000000,114428,StateServices,Водитель,2020-05-17,3403000100000,Среднее,Не применимо,Российская Федерация,...,Одобрено,Видно всем,2020-05-18,false,61,"[A, B, C, D, E]",NaN,NaN,NaN,NaN
4,8b35ebb0-7f41-11ea-8d16-ef76bd2a03c1,3500000000000,NaN,Medicine,Медсестра,2020-04-15,3500000200000,NaN,Не применимо,Российская Федерация,...,Одобрено,Видно всем,2021-07-31,false,53,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5247ffc0-8e20-11eb-8e20-6db06c9eaf56,8900000000000,277288,StateServices,Специалист,2021-03-26,8900000500000,Высшее,Не применимо,Российская Федерация,...,Одобрено,Видно всем,2021-10-12,false,84,NaN,NaN,NaN,NaN,<p>&nbsp;Классный чин- референт муниципальной ...
996,a9c0b7e0-01fb-11ea-9c5f-736ab11edb0c,5600000000000,NaN,Sales,Охранник,2019-11-08,5600000500000,NaN,Не применимо,Российская Федерация,...,Одобрено,Видно всем,2021-07-29,false,48,NaN,NaN,NaN,NaN,NaN
997,91d213b0-04bf-11ea-9846-ef76bd2a03c1,3500000000000,241106,MechanicalEngineering,инженер,2019-11-11,3500100000900,Высшее,Не применимо,Российская Федерация,...,Одобрено,Видно всем,2019-12-12,false,80,[B],<p>Опыт руководящей работы на машиностроительн...,"<p>Настойчивость, целеустремленность.</p>",NaN,NaN
998,7db894f0-cb37-11ea-ba19-ab5d2eb93a75,1800000000000,NaN,Food,Техник-технолог,2020-07-21,1801300500000,NaN,Не применимо,Российская Федерация,...,Одобрено,Видно всем,2020-07-21,false,49,NaN,NaN,NaN,NaN,NaN


Общая информация по датафрейму

In [21]:
data = []
for column in df.columns.tolist():
    l = [column, df[column].isna().sum(), round(df[column].isna().sum() / df.shape[0] * 100, 1), df[column].notna().sum(), df[column].nunique()]
    data.append(l)
dfinfo = pd.DataFrame(data, columns = ['Name', 'NA', 'NA_share', 'Not_NA', 'Unique_values_count'])
dfinfo

,Name,NA,NA_share,Not_NA,Unique_values_count
0,about,0,0.0,1000,1000
1,region_code,0,0.0,1000,81
2,profession_code,560,56.0,440,140
3,industry_code,0,0.0,1000,34
4,positionName,0,0.0,1000,588
5,creationDate,0,0.0,1000,506
6,locality,1,0.1,999,514
7,educationType,241,24.1,759,4
8,worldskills_inspection_status,0,0.0,1000,2
9,country_country_name,0,0.0,1000,3
